# Shortest Path Algorithms: Dijkstra and Bellman-Ford 

### Introduction
In graph theory and computer science, finding the shortest path between nodes is a fundamental problem with a wide range of applications. Two of the most commonly used algorithms to solve this problem are **Dijkstra's Algorithm** and **Bellman-Ford Algorithm**.

### Dijkstra’s Algorithm
- Finds the shortest path from a starting node to all other nodes in a weighted graph.
- **Constraint**: Works with **non-negative weights only**.
- **Efficiency**: Fast for sparse graphs due to priority queue (min-heap).
- **Time Complexity**: O((V + E) log V)

### Bellman-Ford Algorithm
- Also finds shortest paths from a starting node to all other nodes.
- **Advantage**: Handles **negative weights** and detects **negative weight cycles**.
- **Time Complexity**: O(V * E)

### Real-World Applications
- **Dijkstra**: GPS Navigation (fastest routes), Network Routing (OSPF protocol), Games (pathfinding AI).
- **Bellman-Ford**: Currency Arbitrage Detection (negative weights), Routing Protocols (RIP), Financial Networks.

In this notebook, we will implement both algorithms, compare their performance across graphs of different sizes, and visualize their execution times.

In [12]:
import numpy as np
import pandas as pd
import csv
import time
import heapq
from collections import defaultdict
import networkx as nx
import osmnx as ox

In [2]:
# Function to generate and export a sample graph
def generate_and_export_graph(node_count, edge_count, file_path='graph_edges.csv', allow_negative=False):
    G = nx.gnm_random_graph(n=node_count, m=edge_count, directed=True)
    for (u, v) in G.edges():
        weight = np.random.randint(-10, 11) if allow_negative else np.random.randint(1, 11)
        G.edges[u, v]['weight'] = weight

    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Source', 'Target', 'Weight'])
        for u, v, data in G.edges(data=True):
            writer.writerow([u, v, data['weight']])

    return file_path

In [3]:
# Load graph from CSV
def load_graph_from_csv(file_path):
    graph = defaultdict(list)
    nodes = set()
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            u = int(row['Source'])
            v = int(row['Target'])
            w = int(row['Weight'])
            graph[u].append((v, w))
            nodes.update([u, v])
    return graph, list(nodes)

In [4]:
# Dijkstra's Algorithm from scratch
def dijkstra(graph, start):
    dist = {node: float('inf') for node in graph}
    dist[start] = 0
    pq = [(0, start)]
    
    while pq:
        current_dist, u = heapq.heappop(pq)
        if current_dist > dist[u]:
            continue
        for v, weight in graph[u]:
            distance = current_dist + weight
            if distance < dist.get(v, float('inf')):
                dist[v] = distance
                heapq.heappush(pq, (distance, v))
    return dist

In [5]:
# Bellman-Ford Algorithm from scratch
def bellman_ford(graph, nodes, start):
    dist = {node: float('inf') for node in nodes}
    dist[start] = 0

    for _ in range(len(nodes) - 1):
        for u in graph:
            for v, weight in graph[u]:
                if dist[u] + weight < dist[v]:
                    dist[v] = dist[u] + weight
    return dist

In [6]:
# Timing function for custom algorithms
def measure_algorithm_time(func, *args, runs=10):
    times = []
    for _ in range(runs):
        start_time = time.time()
        func(*args)
        end_time = time.time()
        times.append((end_time - start_time) * 1000)  # ms
    return {
        'min': np.min(times),
        'max': np.max(times),
        'avg': np.mean(times),
        'std': np.std(times)
    }

In [7]:
# Graph sizes and settings
graph_settings = [
    {'nodes': 10, 'edges': 20, 'name': 'Small', 'allow_negative': False},
    {'nodes': 100, 'edges': 300, 'name': 'Medium', 'allow_negative': False},
    {'nodes': 1000, 'edges': 3000, 'name': 'Large', 'allow_negative': False},
    {'nodes': 10, 'edges': 20, 'name': 'Small-Negative', 'allow_negative': True},
]

# Store results
table_data = []

In [8]:
for setting in graph_settings:
    print(f"\nRunning for graph: {setting['name']}")
    file_path = generate_and_export_graph(setting['nodes'], setting['edges'], f"graph_{setting['name']}.csv", setting['allow_negative'])
    graph, nodes = load_graph_from_csv(file_path)

    if not setting['allow_negative']:
        dijkstra_results = measure_algorithm_time(dijkstra, graph, nodes[0])
        table_data.append({
            'Graph': setting['name'],
            'Algorithm': 'Dijkstra',
            'Min [ms]': dijkstra_results['min'],
            'Max [ms]': dijkstra_results['max'],
            'Avg [ms]': dijkstra_results['avg'],
            'Std [ms]': dijkstra_results['std']
        })

    bellman_results = measure_algorithm_time(bellman_ford, graph, nodes, nodes[0])
    table_data.append({
        'Graph': setting['name'],
        'Algorithm': 'Bellman-Ford',
        'Min [ms]': bellman_results['min'],
        'Max [ms]': bellman_results['max'],
        'Avg [ms]': bellman_results['avg'],
        'Std [ms]': bellman_results['std']
    })


Running for graph: Small

Running for graph: Medium

Running for graph: Large

Running for graph: Small-Negative


In [9]:
# Display results
df = pd.DataFrame(table_data)
print(df)

            Graph     Algorithm    Min [ms]    Max [ms]    Avg [ms]   Std [ms]
0           Small      Dijkstra    0.000000    0.000000    0.000000   0.000000
1           Small  Bellman-Ford    0.000000    0.000000    0.000000   0.000000
2          Medium      Dijkstra    0.000000    0.997305    0.099730   0.299191
3          Medium  Bellman-Ford    1.998425    3.001451    2.500081   0.500274
4           Large      Dijkstra    0.992775    2.001286    1.498580   0.495416
5           Large  Bellman-Ford  355.102777  584.552050  409.016275  70.856512
6  Small-Negative  Bellman-Ford    0.000000    0.000000    0.000000   0.000000


In [10]:
# Save results to CSV
df.to_csv('performance_results.csv', index=False)

In [11]:
!pip install osmnx

In [16]:
# Function to extract and save real-world graph
def extract_real_world_graph(place_name="Wrocław, Poland"):
    print(f"\nExtracting real-world graph for: {place_name}")
    G_osm = ox.graph_from_place(place_name, network_type='drive')
    G_simplified = G_osm.to_undirected()

    edges_osm = []
    for u, v, data in G_simplified.edges(data=True):
        weight = data.get('length', 1)
        edges_osm.append((u, v, weight))

    real_csv = 'real_road_network.csv'
    with open(real_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Source', 'Target', 'Weight'])
        for u, v, w in edges_osm:
            writer.writerow([u, v, int(w)])

    return real_csv

In [17]:
# Function to run performance tests on real-world graph
def run_real_world_test(place_name="Wrocław, Poland"):
    real_csv = extract_real_world_graph(place_name)
    graph, nodes = load_graph_from_csv(real_csv)

    dijkstra_real = measure_algorithm_time(dijkstra, graph, nodes[0])
    bellman_real = measure_algorithm_time(bellman_ford, graph, nodes, nodes[0])

    return [
        {
            'Graph': f'Real-World ({place_name})',
            'Algorithm': 'Dijkstra',
            'Min [ms]': dijkstra_real['min'],
            'Max [ms]': dijkstra_real['max'],
            'Avg [ms]': dijkstra_real['avg'],
            'Std [ms]': dijkstra_real['std']
        },
        {
            'Graph': f'Real-World ({place_name})',
            'Algorithm': 'Bellman-Ford',
            'Min [ms]': bellman_real['min'],
            'Max [ms]': bellman_real['max'],
            'Avg [ms]': bellman_real['avg'],
            'Std [ms]': bellman_real['std']
        }
    ]

In [20]:
# Real-world graph test
real_data = run_real_world_test()
df_real = pd.DataFrame(real_data)
print("\nReal-World Graph Results:")
print(df_real)
df_real.to_csv('performance_results_real_world.csv', index=False)


Extracting real-world graph for: Wrocław, Poland

Real-World Graph Results:
                          Graph     Algorithm      Min [ms]      Max [ms]  \
0  Real-World (Wrocław, Poland)      Dijkstra      0.000000      1.082182   
1  Real-World (Wrocław, Poland)  Bellman-Ford  10763.916731  21008.255243   

       Avg [ms]     Std [ms]  
0      0.900340     0.302489  
1  13819.186759  3139.594899  
